In [ ]:
import numpy as np

In [23]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = [12, 12]


def fix_scaling(ax=None):
    if not ax:
        xlim = plt.xlim()
        ylim = plt.ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            plt.ylim((ylim[0] - (d1 - d2) / 2, ylim[1] + (d1 - d2) / 2))
        else:
            plt.xlim((xlim[0] + (d1 - d2) / 2, xlim[1] - (d1 - d2) / 2))
    else:
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            ax.set_ylim((ylim[0] - (d1 - d2) / 2, ylim[1] + (d1 - d2) / 2))
        else:
            ax.set_xlim((xlim[0] + (d1 - d2) / 2, xlim[1] - (d1 - d2) / 2))


In [3]:
# 1) a
a = 3
b = 6


def func(x):
    return a * x[0] ** 2 + b * (x[0] - x[1]) ** 2 - x[0] - 2 * x[1]


def f_grad(x):
    return np.array([18 * x[0] - 12 * x[1] - 1, 12 * x[1] - 12 * x[0] - 2])


matrix_form = np.array([[18, -12], [-12, 12]])
lambdas = list(np.linalg.eig(matrix_form)[0])

In [4]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML


def animate_trajectory(traj):
    fig, ax = plt.subplots()
    n = len(traj)

    def step(t):
        ax.cla()
        ax.plot([1.0 / 2], [2.0 / 3], 'o', color='green')
        #Level contours
        delta = 0.025
        x = np.arange(-3, 3, delta)
        y = np.arange(-3, 3, delta)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)
        #print(X.shape, Y.shape)
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i][j] = func([X[i][j], Y[i][j]])
        CS = ax.contour(X, Y, Z, [0.5, 1.5, 3], colors=['blue', 'purple', 'red'])

        ax.plot([u[0] for u in traj[:t]], [u[1] for u in traj[:t]], color='black')
        ax.plot([u[0] for u in traj[:t]], [u[1] for u in traj[:t]], 'o', color='black')

        fix_scaling(ax)
        ax.axis('off')

    return FuncAnimation(fig, step,
                         frames=range(n), interval=600)


In [5]:
# 1 b
#Opt size
alpha = 2.0 / (sum(list(np.linalg.eig(matrix_form)[0])))
traj_opt_step = []
x_start = np.array([2, 2.8])
traj_opt_step.append(x_start.copy())
cur_x = x_start.copy()
for i in range(15):
    cur_x = cur_x - alpha * f_grad(cur_x)
    traj_opt_step.append(cur_x.copy())

#print(traj)
base_animation = animate_trajectory(traj_opt_step)
HTML(base_animation.to_html5_video())


In [ ]:

#Heavy ball
alpha = 4.0 / (np.sqrt(lambdas[0]) + np.sqrt(lambdas[1])) ** 2
beta = (np.sqrt(lambdas[0]) - np.sqrt(lambdas[1])) / (np.sqrt(lambdas[0]) + np.sqrt(lambdas[1]))
traj_heavy_ball = []
x_start = np.array([2, 2.8])
traj_heavy_ball.append(x_start.copy())
cur_x = x_start.copy()
prev_x = x_start.copy()
for i in range(15):
    t = cur_x
    cur_x = cur_x - alpha * f_grad(cur_x) + beta * (cur_x - prev_x)
    prev_x = t
    traj_heavy_ball.append(cur_x.copy())

#print(traj)
base_animation = animate_trajectory(traj_heavy_ball)
HTML(base_animation.to_html5_video())


In [ ]:
#Chebyshev
phi = (lambdas[0] + lambdas[1]) / (lambdas[0] - lambdas[1])
cur_gamma = 1.0 / phi
prev_gamma = 0
print(phi - np.sqrt(phi ** 2 - 1))
print('phi', phi)
traj_chebyshev = []
x_start = np.array([2, 2.8])
traj_chebyshev.append(x_start.copy())
cur_x = x_start.copy()
prev_x = x_start.copy()
for i in range(15):
    t = cur_x
    alpha = 4 * cur_gamma / (lambdas[0] - lambdas[1])
    beta = cur_gamma * prev_gamma
    cur_x = cur_x - alpha * f_grad(cur_x) + beta * (cur_x - prev_x)
    prev_x = t
    t = cur_gamma
    cur_gamma = 1.0 / (2 * phi - cur_gamma)
    prev_gamma = t
    print('gamma', cur_gamma)
    traj_chebyshev.append(cur_x.copy())

#print(traj)
base_animation = animate_trajectory(traj_chebyshev)
HTML(base_animation.to_html5_video())


In [8]:
#Nesterov
alpha = 1 / (lambdas[0] + 1)
beta = (np.sqrt(lambdas[0]) - np.sqrt(lambdas[1])) / (np.sqrt(lambdas[0]) + np.sqrt(lambdas[1]))
traj_nesterov = []
x_start = np.array([2, 2.8])
traj_nesterov.append(x_start.copy())
cur_x = x_start.copy()
cur_y = x_start.copy()

for i in range(15):
    t = cur_x
    cur_x = cur_y - alpha * f_grad(cur_y)
    cur_y = cur_x + beta * (cur_x - t)
    traj_nesterov.append(cur_x.copy())
    
base_animation = animate_trajectory(traj_nesterov)
HTML(base_animation.to_html5_video())

In [12]:
fig, ax = plt.subplots()
min_x = [1.0/2,2.0/3]
u = np.array(traj_opt_step)
ax.plot(range(16), [np.linalg.norm(t - min_x) for t in u], label='Optimal step size')
u = np.array(traj_heavy_ball)
ax.plot(range(16), [np.linalg.norm(t - min_x) for t in u], label='Heavy ball')
u = np.array(traj_chebyshev)
ax.plot(range(16), [np.linalg.norm(t - min_x) for t in u], label='Chebyshev')
u = np.array(traj_nesterov)
ax.plot(range(16), [np.linalg.norm(t - min_x) for t in u], label='Nesterov')
plt.legend()
plt.show()

In [10]:
# 1 c
matr_a = np.array([[2 * (a + b), -2 * b], [-2 * b, 2 * b]])
matr_b = np.array([1, 2])
print(np.linalg.solve(matr_a, matr_b))

[0.5        0.66666667]


In [11]:
fig, ax = plt.subplots()
f_min = func(min_x)
u = np.array(traj_opt_step)
ax.plot(range(16), [func(t) - f_min for t in u], label='Optimal step size')
u = np.array(traj_heavy_ball)
ax.plot(range(16), [func(t) - f_min for t in u], label='Heavy ball')
u = np.array(traj_chebyshev)
ax.plot(range(16), [func(t) - f_min for t in u], label='Chebyshev')
u = np.array(traj_nesterov)
ax.plot(range(16), [func(t) - f_min for t in u], label='Nesterov')
plt.legend()
plt.show()

In [ ]:
#1 d
def animate_trajectories(trajectories):
    fig, ax = plt.subplots()
    n = len(trajectories[0])
    def step(t):
        ax.cla()
        ax.plot([1.0 / 2], [2.0 / 3], 'o', color='green')
        #Level contours
        delta = 0.025
        x = np.arange(-3, 3, delta)
        y = np.arange(-3, 3, delta)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)
        #print(X.shape, Y.shape)
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i][j] = func([X[i][j], Y[i][j]])
        CS = ax.contour(X, Y, Z, [0.5, 1.5, 3], colors=['blue', 'purple', 'red'])

        colors = ['blue', 'yellow', 'black', 'red']
        labels = ['Optimal step size', 'Heavy ball', 'Chebyshev', 'Nesterov']

        for i in range(4):
            traj = trajectories[i]
            ax.plot([u[0] for u in traj[:t]], [u[1] for u in traj[:t]], color=colors[i], label=labels[i])
            ax.plot([u[0] for u in traj[:t]], [u[1] for u in traj[:t]], 'o', color=colors[i])

        plt.legend()

        fix_scaling(ax)
        ax.axis('off')

    return FuncAnimation(fig, step,
                         frames=range(n), interval=600)


In [ ]:
#print(trajectories)
base_animation = animate_trajectories([traj_opt_step, traj_heavy_ball, traj_chebyshev, traj_nesterov])
HTML(base_animation.to_html5_video())



In [26]:
# 2 a
from math import exp


def func(x):
    return exp(x[0] + 3 * x[1]) + exp(x[0] -3 * x[1]) + exp(-x[0])


def f_grad(x):
    return np.array([exp(x[0] + 3 * x[1]) + exp(x[0] -3 * x[1]) - exp(-x[0]),
                     3 * exp(x[0] + 3 * x[1]) - 3 * exp(x[0] - 3 * x[1])])

In [37]:
# 2 b


def lipschitz_approx():
    delta = 0.1
    x = np.arange(-1, 1, delta)
    y = np.arange(-1, 1, delta)
    approx = 0
    for i in x:
        for j in y:
            for i2 in x:
                for j2 in y:
                    if i != i2 or j != j2:
                        approx = max(approx, abs(
                            np.linalg.norm(f_grad([i, j]) - f_grad([i2, j2])) / np.linalg.norm([i - i2, j - j2])))
    return approx * 2


print(lipschitz_approx())


811.978943248


In [38]:
# 2 c

#Nesterov
coef = 1.0 / lipschitz_approx()
alpha = 0.9
traj_nesterov = []
x_start = np.array([1.0 / 2 , 1.0 / 2])
traj_nesterov.append(x_start.copy())
cur_x = x_start.copy()
cur_y = x_start.copy()

for i in range(200):
    t = cur_x
    cur_x = cur_y - coef * f_grad(cur_y)
    cur_alpha = (-alpha + np.sqrt(4 * alpha + alpha ** 2)) / 2
    beta = (alpha * (1 - alpha)) / (alpha ** 2 + cur_alpha)
    cur_y = cur_x + beta * (cur_x - t)
    traj_nesterov.append(cur_x.copy())
    alpha = cur_alpha
    
base_animation = animate_trajectory(traj_nesterov)
HTML(base_animation.to_html5_video())

NameError: name 'animate_trajectory' is not defined

In [ ]:
#Opt size
alpha = 2.0 / lipschitz_approx()
traj_opt_step = []
x_start = np.array([1.0 / 2, 1.0 / 2])
traj_opt_step.append(x_start.copy())
cur_x = x_start.copy()
for i in range(200):
    cur_x = cur_x - alpha * f_grad(cur_x)
    traj_opt_step.append(cur_x.copy())

#print(traj)
base_animation = animate_trajectory(traj_opt_step)
HTML(base_animation.to_html5_video())

In [ ]:
# 3 a


def poly_regression(x, y, n):
    return np.polyfit(x, y, n)

In [11]:
# 3 b
from numpy import *
from scipy import optimize


def least_squares_circle(x, y):
    x_m = mean(x)
    y_m = mean(y)

    def calc_R(xc, yc):
        return sqrt((x - xc) ** 2 + (y - yc) ** 2)

    def f_2(c):
        Ri = calc_R(c[0], c[1])
        return Ri - Ri.mean()

    center_estimate = x_m, y_m
    center, _ = optimize.leastsq(f_2, center_estimate)

    xc, yc = center
    Ri = calc_R(xc, yc)
    R = Ri.mean()
    return [xc, yc, R]


least_squares_circle(r_[36, 36, 19, 18, 33, 26], r_[14, 10, 28, 31, 18, 26])


4.042521898337994


[9.887594987736426, 3.6875233988265963, 27.35040767420324]

In [24]:
fig, ax = plt.subplots()
ax.plot([u for u in r_[36, 36, 19, 18, 33, 26]], [u for u in r_[14, 10, 28, 31, 18, 26]], 'o', color='green')
x, y, r = least_squares_circle(r_[36, 36, 19, 18, 33, 26], r_[14, 10, 28, 31, 18, 26])
circle = plt.Circle((x, y), r, color='b', fill=False)
fix_scaling(ax)
ax.axis('off')
ax.add_artist(circle)
plt.show()

4.04252189834
